In [1]:
import numpy as np
import copy
import time
import pytz
import datetime
import math
import re
import os
import pandas as pd
from pandas import read_csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn import datasets, linear_model
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from scipy import stats
# whether or not to split into train and test
split_data = True

# New Models

In [2]:

'''

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

dataset = pd.read_csv('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv')
dataset.head()

X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

dX = pd.DataFrame(X,columns=dataset.columns[:-1])

X1 = dX.values

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)

from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

print(regressor.coef_)

print(regressor.intercept_)

y_pred = regressor.predict(X_test)
y_pred 
'''

"\n\nimport numpy as np\nimport matplotlib.pyplot as plt\nimport pandas as pd\n\ndataset = pd.read_csv('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv')\ndataset.head()\n\nX = dataset.iloc[:, :-1].values\ny = dataset.iloc[:, -1].values\n\ndX = pd.DataFrame(X,columns=dataset.columns[:-1])\n\nX1 = dX.values\n\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X1, y, test_size = 0.2, random_state = 0)\n\nfrom sklearn.linear_model import LinearRegression\nregressor = LinearRegression()\nregressor.fit(X_train, y_train)\n\nprint(regressor.coef_)\n\nprint(regressor.intercept_)\n\ny_pred = regressor.predict(X_test)\ny_pred \n"

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

#dataset = pd.read_csv('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv')


def getTrainTest(csv, labels, disease):
    #print(csv)
    #print(csv.loc[:, labels])

    csv = csv.loc[:, labels]

    #csv = csv[0][features]
    #newDataset = csv[:,features]
    X = csv.iloc[:, :-1].values
    #print(X)
    y = csv.iloc[:, -1].values    # split into train and test sets
    #print(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    features = y
    
    return X_train, X_test, y_train, y_test


def splitByDis(csv, dis):
    csvSubset = pd.DataFrame()
        
    new_column_name = "Disease (CF=1,H=2,copd=3)"
    new_val = csv.columns.get_loc(new_column_name)

    index = new_val
    for row in csv.iterrows():
        if row[1][index] == float(dis):
            #print(row[1])
            csvSubset = csvSubset.append(row[1])
            #print(csvSubset)
    csvSubset = pd.DataFrame(data=csvSubset)
    #print(csvSubset)
    return csvSubset

def convertLabels(csv, features):
    data = read_csv(csv, header=None)
    dataset = data.values
    #print(dataset.shape)
    #print(dataset[:,features])
    labels = dataset[0][features]
    
    #newDataset = dataset[:,features]
    
    #print(features)
    #print(labels)
    
    return labels


def predictions(csv, feats, disease):
    
    dataset = pd.read_csv(csv)
    dataset = splitByDis(dataset, disease)
    
    labels = convertLabels(csv, feats)
    X_train, X_test, y_train, y_test = getTrainTest(dataset, labels, disease)
    
    #model = sm.OLS(y_train, X_train).fit() ## sm.OLS(output, input)

    
    model = LinearRegression()
    #model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    #print(model.summary())

    y_pred = model.predict(X_test)
    pred = np.linalg.norm(y_pred - np.array(y_test).astype(float))
    #print(pred)
    #print(y_test)
    corr = correlation(y_pred, y_test)
    return(pred, y_test, corr)

import seaborn as sns
import matplotlib.pyplot as plt


def correlation(y_pred, y_test):
    covariance = np.cov(y_pred, y_test)
    #print("Covariance matrix of x:\n", np.cov(a))
    #print("\nCovariance matrix of the said arrays:\n", covariance)

    pred = ['y_pred', 'y_test']
    
    correlation = np.corrcoef(y_pred, y_test)
    #print(correlationArray(correlation))
    return(correlation[0][1])
    #sns.heatmap(covariance, annot=True, fmt='g', xticklabels=pred, yticklabels=pred)
    #plt.show()
    
    # plotting correlation heatmap
    #dataplot = sns.heatmap(correlation, cmap="YlGnBu", annot=True, vmax=1, vmin=-1, xticklabels=pred, yticklabels=pred)
  
    # displaying heatmap
    #plt.show()
    
def correlationArray(correlation):
    return correlation[0][1]
    
def bestFeats(csv, feats, disease):
    featurePred=[]
    featureCorrs=[]
    featsCopy = feats
    length = len(feats)
    
    for x in range(length-1):
        featsCopy = featsCopy[:x] + featsCopy[(x+1):]
        predict = predictions(csv, featsCopy, disease)
        featurePred.append(predict[0])
        featureCorrs.append(predict[2])
        featsCopy = []
        featsCopy = feats   
        
    print("Prediction")
    print(featurePred)
    print(predict[1])

    #featurePred shows the prediction if this feature was removed. If score is low, this means prediction would be lower if this was removed
    
    L = [ (featurePred[i],i) for i in range(len(featurePred)) ]
    L.sort()
    sorted_l,permutationL = zip(*L)
    permutationNpL = np.array(permutationL)
    print(permutationNpL)
    print(convertLabels(csv, permutationNpL))
    #featureCorrs shows the prediction if this feature was removed. If score is low, this means prediction would be lower if this was removed

    
    print("Correlation")
    print(featureCorrs)

    N = [ (featureCorrs[j],j) for j in range(len(featureCorrs)) ]
    N.sort()
    sorted_n,permutationN = zip(*N)
    permutationNpN = np.array(permutationN)
        
    print(permutationNpN)
    print(convertLabels(csv, permutationNpN))

    featurePred = np.array(featurePred)
   

    print("variance: ", np.var(featureCorrs))

    return permutationNpL, permutationNpN

#predsCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 1)
#print(convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsCOPD[0]))
#print(convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsCOPD[1]))



# Fatty

Comb Dia Fatty

In [4]:
features = [0,1,2,3,4,5,6,7]

In [5]:
predsDiaFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 1)
#print(convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFCF))
print("-------------")
predsDiaFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 2)
#print(convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFHealthy))
print("-------------")
predsDiaFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', features, 3)
#print(convertLabels('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combDiaFatty.csv', predsDiaFCOPD))
print("-------------")

Prediction
[23.954962020637268, 23.93922641891992, 24.12384360045828, 24.520673647394858, 25.63040539532676, 24.891463234941156, 24.234855851355498]
[42.47 36.83 45.5  55.4  28.57 43.14 36.22 28.01 38.57]
[1 0 2 6 3 5 4]
['Age' 'Gender(1=F)' 'BMI' 'Disease (CF=1,H=2,copd=3)' 'Sub' 'FEV1%' 'Vis']
Correlation
[0.5600174847286669, 0.5572153808914947, 0.5749675394683829, 0.5705316655879499, 0.2813536811083175, 0.5895608792403082, 0.568306584133312]
[4 1 0 6 3 2 5]
['Vis' 'Age' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'Sub' 'BMI' 'FEV1%']
variance:  0.010305322340470265
-------------
Prediction
[16.178842506972785, 15.29921647377633, 17.751987714116012, 14.736203366254472, 19.346096905502023, 17.042238417294087, 15.268695806043832]
[43.83 55.21 48.94 35.8  42.88 54.21 54.89]
[3 6 1 0 5 2 4]
['Sub' 'Disease (CF=1,H=2,copd=3)' 'Age' 'Gender(1=F)' 'FEV1%' 'BMI' 'Vis']
Correlation
[0.7786979089986925, 0.7967328785608243, 0.6554695327455743, 0.7863866226767174, 0.05312060009703143, 0.4781330161

In [6]:
predsPecsFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 1)
print("-------------")
predsPecsFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 2)
print("-------------")
predsPecsFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combPecsFatty.csv', features, 3)
print("-------------")

Prediction
[40.7990530140876, 31.370539277829852, 32.79439567541077, 35.07256452688705, 39.25655161331982, 34.872920709258274, 32.60263241637564]
[19.57 42.52 35.49 59.   15.73 18.15 22.01 15.78 17.59]
[1 6 2 5 3 4 0]
['Age' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'FEV1%' 'Sub' 'Vis' 'Gender(1=F)']
Correlation
[0.34275907048458265, 0.7292553120928807, 0.7295792360707828, 0.6448297314703745, 0.4438295419634945, 0.6624456475395205, 0.702301235980601]
[0 4 3 5 6 1 2]
['Gender(1=F)' 'Vis' 'Sub' 'FEV1%' 'Disease (CF=1,H=2,copd=3)' 'Age' 'BMI']
variance:  0.02000034131125693
-------------
Prediction
[31.259198437957302, 27.27819335801636, 28.100887729318583, 30.746090009754976, 29.641160230928328, 28.550454955120674, 29.657698784131007]
[35.69 50.48 30.09 33.93 39.01 47.53 26.37]
[1 2 5 4 6 3 0]
['Age' 'BMI' 'FEV1%' 'Vis' 'Disease (CF=1,H=2,copd=3)' 'Sub' 'Gender(1=F)']
Correlation
[-0.052100738008495684, 0.5615602922589971, 0.4333060143587857, 0.6476015761292223, 0.5793186944776886, 0.64514926358

In [7]:
predsParaFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 1)
print("-------------")
predsParaFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 2)
print("-------------")
predsParaFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combParaFatty.csv', features, 3)
print("-------------")

Prediction
[46.44657742506111, 27.554660126237366, 37.99719490242103, 36.77484223762354, 37.24688803910969, 30.407933633288742, 37.94162079606097]
[18.72 20.99 22.82 47.2  11.79 17.33 25.25 10.04 24.5 ]
[1 5 3 4 6 2 0]
['Age' 'FEV1%' 'Sub' 'Vis' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Gender(1=F)']
Correlation
[-0.5227015912327785, 0.7639289367287504, 0.25940808076722566, 0.3216862934179975, 0.31522576285110343, 0.5533215323390014, 0.24571847995275542]
[0 6 2 4 3 5 1]
['Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Vis' 'Sub' 'FEV1%' 'Age']
variance:  0.1368173235867842
-------------
Prediction
[52.713363641627474, 54.91512857099811, 47.88787710722547, 65.45000068681308, 50.79572616823038, 47.834131575769206, 51.450124944987564]
[34.1  42.62 36.64 33.53 35.74 51.85 33.96]
[5 2 4 6 0 1 3]
['FEV1%' 'BMI' 'Vis' 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)' 'Age' 'Sub']
Correlation
[0.3149694970429295, 0.33045867750589847, 0.048798643373222, 0.6399857727987376, 0.1805647883031071, 0.2688960418169

In [8]:
predsAbdFCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 1)
print("-------------")
predsAbdFHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 2)
print("-------------")
predsAbdFCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/fatty/combAbdFatty.csv', features, 3)
print("-------------")

Prediction
[42.3596087953601, 31.38168562260483, 40.0386966316781, 38.50712273996456, 36.95458581567462, 42.52544310288632, 40.32010485212029]
[39.61 36.97 56.02 71.35 29.36 42.19 47.1  29.33 38.51]
[1 4 3 2 6 0 5]
['Age' 'Vis' 'Sub' 'BMI' 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)' 'FEV1%']
Correlation
[0.009574474023372909, 0.754606523968775, 0.5156437342207632, 0.5372990358185793, 0.6815134676126055, 0.2642630134901144, 0.39193746875394264]
[0 5 6 2 3 4 1]
['Gender(1=F)' 'FEV1%' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Sub' 'Vis' 'Age']
variance:  0.05573637541324556
-------------
Prediction
[35.058838064307956, 31.337949273954965, 33.54350481194867, 32.5878194706003, 39.3189540571723, 27.652763529814028, 35.391790299464006]
[40.28 45.86 61.39 35.11 44.17 50.9  42.2 ]
[5 1 3 2 0 6 4]
['FEV1%' 'Age' 'Sub' 'BMI' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'Vis']
Correlation
[-0.10601262341376232, -0.08617959449267554, 0.07137208523031172, 0.1032664044917293, -0.10406690655213162, -0.07220470068

In [9]:
features = [0,1,2,3,4,5,6]

# Muscle

In [10]:
predsDiaMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 1)
print("-------------")
predsDiaMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 2)
print("-------------")
predsDiaMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combDiaM.csv', features, 3)
print("-------------")

Prediction
[52.50037508186201, 45.17519699489604, 42.90422212445816, 46.55445504335789, 46.141137431703754, 54.999128685740224]
[ 71.56  85.5   87.36  96.57 113.2  102.13  99.04 103.42  97.37]
[2 1 4 3 0 5]
['Height' 'Age' 'FEV1%' 'BMI' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)']
Correlation
[0.32660867794490644, 0.46683033761829706, 0.494927157797994, 0.32052338577032496, 0.49050436590256846, 0.42604248246292376]
[3 0 5 1 4 2]
['BMI' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'Age' 'FEV1%' 'Height']
variance:  0.0052378577936921175
-------------
Prediction
[36.476727879472136, 40.005615912070496, 37.42860006051678, 34.23527576819515, 48.73405465062161, 37.08934421194229]
[67.7684 56.9269 64.7289 39.0636 40.0654 50.7596 29.99  ]
[3 0 5 2 1 4]
['BMI' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'Height' 'Age' 'FEV1%']
Correlation
[0.4105441403262274, 0.3303349852332617, 0.4751227222480227, 0.4834749563829727, 0.1307121336676992, 0.4698084723674198]
[4 1 0 5 2 3]
['FEV1%' 'Age' 'Gender(1=F)' 

In [11]:
predsPecsMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 1)
print("-------------")
predsPecsMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 2)
print("-------------")
predsPecsMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combPecsM.csv', features, 3)
print("-------------")

Prediction
[44.21408116567522, 48.785321404849604, 43.57581545417709, 42.443983902882565, 40.81816983195654, 41.771663719450885]
[15.101 35.627 48.217 33.121 53.075 30.119 36.323 39.225 35.386]
[4 5 3 2 0 1]
['FEV1%' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Height' 'Gender(1=F)' 'Age']
Correlation
[-0.4793830579455198, 0.07559454420512875, -0.2744695402078185, -0.4609189509941178, -0.5133666996570402, -0.4668681257212878]
[4 0 5 3 2 1]
['FEV1%' 'Gender(1=F)' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Height' 'Age']
variance:  0.0426937649933777
-------------
Prediction
[31.948167870862182, 31.908201645299055, 30.394261218260063, 28.621334066853112, 30.43455942489063, 27.33223252435528]
[34.3  31.2  42.2  24.17 33.2  27.14 29.2 ]
[5 3 2 4 1 0]
['Disease (CF=1,H=2,copd=3)' 'BMI' 'Height' 'FEV1%' 'Age' 'Gender(1=F)']
Correlation
[0.02732951240192682, 0.4414048547365654, 0.35503132139919313, 0.5994706531934915, 0.45869127347736144, 0.5163105991808755]
[0 2 1 4 5 3]
['Gender(1=F)' 'Height' 'Age' 'FEV1%' 

In [12]:
predsParaMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 1)
print("-------------")
predsParaMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 2)
print("-------------")
predsParaMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combParaM.csv', features, 3)
print("-------------")

Prediction
[26.178790333072914, 18.906003713071133, 20.608696197997936, 25.6112973866624, 23.732809257538136, 25.578361903491196]
[26.5  16.82 34.   30.   49.2  32.9  26.1  33.75 29.8 ]
[1 2 4 5 3 0]
['Age' 'Height' 'FEV1%' 'Disease (CF=1,H=2,copd=3)' 'BMI' 'Gender(1=F)']
Correlation
[0.18017996744725936, 0.8181805188392671, 0.6901814155881055, 0.27491667341907816, 0.5238049232687425, 0.3515208992961599]
[0 3 5 4 2 1]
['Gender(1=F)' 'BMI' 'Disease (CF=1,H=2,copd=3)' 'FEV1%' 'Height' 'Age']
variance:  0.05143935719767351
-------------
Prediction
[20.43440416107941, 20.118102737978855, 21.712655698059848, 17.827779742662244, 22.762137449175317, 20.316469220307916]
[34.9  18.3  26.2  16.7  25.85 16.2  19.2 ]
[3 1 5 0 2 4]
['BMI' 'Age' 'Disease (CF=1,H=2,copd=3)' 'Gender(1=F)' 'Height' 'FEV1%']
Correlation
[0.20079319330594747, 0.2857803810619064, 0.4349195122756278, 0.11483767327835978, 0.1862592293695819, 0.29359383258342037]
[3 4 0 1 5 2]
['BMI' 'FEV1%' 'Gender(1=F)' 'Age' 'Disease (CF=

In [13]:
predsAbdMCF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 1)
print("-------------")
predsAbdMHealthy = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 2)
print("-------------")
predsAbdMCOPD = bestFeats('/Users/soniasharapova/Documents/FatiResearch/combinedCSV/muscle/combAbdM.csv', features, 3)
print("-------------")

Prediction
[30.055717856475994, 36.83552035202289, 39.1218010449767, 38.76946225785183, 47.73117942188278, 41.278941356197386]
[51.68 49.5  60.51 54.1  80.22 58.57 48.53 60.9  59.2 ]
[0 1 3 2 5 4]
['Gender(1=F)' 'Age' 'BMI' 'Height' 'Disease (CF=1,H=2,copd=3)' 'FEV1%']
Correlation
[0.45777130478554157, 0.5468828006228927, 0.5413423081620191, 0.48280224037717345, 0.5204660540740692, 0.5777490551854697]
[0 3 4 2 1 5]
['Gender(1=F)' 'BMI' 'FEV1%' 'Height' 'Age' 'Disease (CF=1,H=2,copd=3)']
variance:  0.0016268724474818367
-------------
Prediction
[27.976875866722754, 38.37368038422076, 40.38025461245646, 47.65274353627504, 42.155438275824466, 41.51954364737239]
[46.43 54.4  36.   31.9  51.8  44.2  58.3 ]
[0 1 2 5 4 3]
['Gender(1=F)' 'Age' 'Height' 'Disease (CF=1,H=2,copd=3)' 'FEV1%' 'BMI']
Correlation
[0.18833912809810022, -0.3419725654287832, -0.41183109470992113, -0.5642030629255976, -0.4434674472950646, -0.44022722682714155]
[3 4 5 2 1 0]
['BMI' 'FEV1%' 'Disease (CF=1,H=2,copd=3)' 'Hei

In [18]:
features = [0,1,2,3,4,5,6,7]
print(features[:-1])

[0, 1, 2, 3, 4, 5, 6]


In [15]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm

features = [0,1,2,3,4,5,6,7]


def getTrainTest(csv, labels):
    #print(csv)
    #print(csv.loc[:, labels])

    csv = csv.loc[:, labels]

    #csv = csv[0][features]
    #newDataset = csv[:,features]
    X = csv.iloc[:, :-1].values
    #print(X)
    y = csv.iloc[:, -1].values    # split into train and test sets
    #print(y)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, shuffle=False)
    features = y
    
    return X_train, X_test, y_train, y_test


def convertLabels(csv, features):
    data = read_csv(csv, header=None)
    dataset = data.values
    #print(dataset.shape)
    #print(dataset[:,features])
    labels = dataset[0][features]
    
    #newDataset = dataset[:,features]
    
    #print(features)
    #print(labels)
    
    return labels


def predictions(csv, feats):
    
    dataset = pd.read_csv(csv)
    
    labels = convertLabels(csv, feats)
    X_train, X_test, y_train, y_test = getTrainTest(dataset, labels)
    
    #model = sm.OLS(y_train, X_train).fit() ## sm.OLS(output, input)

    model = LinearRegression()
    #model = RandomForestRegressor()
    model.fit(X_train, y_train)
    
    #print(model.summary())

    y_pred = model.predict(X_test)
    pred = np.linalg.norm(y_pred - np.array(y_test).astype(float))
    #print(pred)
    #print(y_test)
    corr = correlation(y_pred, y_test)
    return(pred, y_test, corr)

import seaborn as sns
import matplotlib.pyplot as plt


def correlation(y_pred, y_test):
    covariance = np.cov(y_pred, y_test)
    #print("Covariance matrix of x:\n", np.cov(a))
    #print("\nCovariance matrix of the said arrays:\n", covariance)

    pred = ['y_pred', 'y_test']
    
    correlation = np.corrcoef(y_pred, y_test)
    #print(correlationArray(correlation))
    return(correlation[0][1])
    #sns.heatmap(covariance, annot=True, fmt='g', xticklabels=pred, yticklabels=pred)
    #plt.show()
    
    # plotting correlation heatmap
    #dataplot = sns.heatmap(correlation, cmap="YlGnBu", annot=True, vmax=1, vmin=-1, xticklabels=pred, yticklabels=pred)
  
    # displaying heatmap
    #plt.show()
    
def correlationArray(correlation):
    return correlation[0][1]
    
def bestFeats(csv, feats):
    featurePred=[]
    featureCorrs=[]
    featsCopy = feats
    length = len(feats)
    
    for x in range(length-1):
        featsCopy = featsCopy[:x] + featsCopy[(x+1):]
        predict = predictions(csv, featsCopy)
        featurePred.append(predict[0])
        featureCorrs.append(predict[2])
        featsCopy = []
        featsCopy = feats   
        
    print("Prediction")
    print(featurePred)

    #featurePred shows the prediction if this feature was removed. If score is low, this means prediction would be lower if this was removed
    
    L = [ (featurePred[i],i) for i in range(len(featurePred)) ]
    L.sort()
    sorted_l,permutationL = zip(*L)
    permutationNpL = np.array(permutationL)
    print(permutationNpL)
    print(convertLabels(csv, permutationNpL))
    #featureCorrs shows the prediction if this feature was removed. If score is low, this means prediction would be lower if this was removed

    
    print("Correlation")
    print(featureCorrs)

    N = [ (featureCorrs[j],j) for j in range(len(featureCorrs)) ]
    N.sort()
    sorted_n,permutationN = zip(*N)
    permutationNpN = np.array(permutationN)
        
    print(permutationNpN)
    print(convertLabels(csv, permutationNpN))

    featurePred = np.array(featurePred)
   
    return permutationNpL, permutationNpL


In [16]:
predsDiaF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Fatty/RCFDiaF.csv', features)
print("-------------")
predsPecsF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Fatty/RCFPecsF.csv', features)
print("-------------")
predsParaF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Fatty/RCFParaF.csv', features)
print("-------------")
predsAbdF = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Fatty/RCFAbdF.csv', features)
print("-------------")

Prediction
[29.210221307074118, 29.53670393159861, 33.323585753323606, 31.976400843279702, 31.62588574065353, 35.293571946484676, 35.47254940447769]
[0 1 4 3 2 5 6]
['Gender(1=F)' 'Age' 'BMI' 'Weight' 'Height' 'FEV1%' 'FVC % ']
Correlation
[0.17885377607018668, 0.05559615677813806, -0.04364736705164505, 0.020144771860083097, 0.03170867131636375, -0.0422711121747384, -0.08241042459597808]
[6 2 5 3 4 1 0]
['FVC % ' 'Height' 'FEV1%' 'Weight' 'BMI' 'Age' 'Gender(1=F)']
-------------
Prediction
[37.37575432808306, 35.63241644217306, 41.26086721471792, 42.49295293473209, 40.36333782039944, 38.91667846240816, 36.28376784303892]
[1 6 0 5 4 2 3]
['Age' 'FVC % ' 'Gender(1=F)' 'FEV1%' 'BMI' 'Height' 'Weight']
Correlation
[0.5060909717263827, 0.5847260325314646, 0.36937486612213927, 0.30091513623807487, 0.3995200415371514, 0.49427590207913613, 0.585439562877508]
[3 2 4 5 0 1 6]
['Weight' 'Height' 'BMI' 'FEV1%' 'Gender(1=F)' 'Age' 'FVC % ']
-------------
Prediction
[38.83010160145687, 27.1620452039

In [17]:
predsDiaM = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Muscle/RCFDiaM.csv', features)
print("-------------")
predsPecsM = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Muscle/RCFPecsM.csv', features)
print("-------------")
predsParaM = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Muscle/RCFParaM.csv', features)
print("-------------")
predsAbdM = bestFeats('/Users/soniasharapova/Documents/FatiResearch/CFonlyCSV/Muscle/RCFAbdM.csv', features)
print("-------------")

Prediction
[63.14970780378674, 62.01724348456836, 60.243172877639616, 56.54956686659586, 56.80102039033801, 58.42114881428419, 55.34962483745811]
[6 3 4 5 2 1 0]
['FVC % ' 'Weight' 'BMI' 'FEV1%' 'Height' 'Age' 'Gender(1=F)']
Correlation
[0.2998002230722084, 0.34598642165038, 0.31360877355872985, 0.35792457799095223, 0.3576650380137193, 0.33787365392718194, 0.3892267713211349]
[0 2 5 1 4 3 6]
['Gender(1=F)' 'Height' 'FEV1%' 'Age' 'BMI' 'Weight' 'FVC % ']
-------------
Prediction
[52.6616747489379, 57.601907341057085, 49.54203137444171, 48.7575367224879, 50.43462538206664, 41.2267988056924, 45.47481770192051]
[5 6 3 2 4 0 1]
['FEV1%' 'FVC % ' 'Weight' 'Height' 'BMI' 'Gender(1=F)' 'Age']
Correlation
[-0.1330296457098925, 0.1898538735808248, 0.229065313068017, -0.06147183425399262, -0.09338657989758685, 0.034839190932608954, 0.09846845786948372]
[0 4 3 5 6 1 2]
['Gender(1=F)' 'BMI' 'Weight' 'FEV1%' 'FVC % ' 'Age' 'Height']
-------------
Prediction
[28.44732549462035, 21.41454346293012, 24.